In [9]:
import cv2
import numpy as np

# Known width of the reference object (e.g., in inches or centimeters)
KNOWN_WIDTH = 2.0  # Adjust based on your reference object

# Function to calculate the midpoint between two points
def midpoint(ptA, ptB):
    return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

# Function to perform camera calibration and undistort the image
def undistort_image(image, mtx, dist):
    h, w = image.shape[:2]
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
    undistorted = cv2.undistort(image, mtx, dist, None, newcameramtx)
    x, y, w, h = roi
    return undistorted[y:y+h, x:x+w]

# Load camera calibration data
# These values should be obtained from camera calibration process
mtx = np.array([[1.15777955e+03, 0.00000000e+00, 6.67111029e+02],
                [0.00000000e+00, 1.15282236e+03, 3.86124583e+02],
                [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
dist = np.array([[-2.42565104e-01, -4.77893070e-02, -1.31388084e-03, -1.54726929e-03, 2.30650029e-02]])

# Initialize video stream
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Undistort the image
    frame = undistort_image(frame, mtx, dist)

    # Convert to grayscale and apply Gaussian blur and Canny edge detection in a single step
    gray_blurred = cv2.GaussianBlur(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), (7, 7), 0)
    edged = cv2.Canny(gray_blurred, 50, 100)
    
    # Find contours
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Sort the contours from left-to-right and filter small contours early
    contours = sorted([c for c in contours if cv2.contourArea(c) > 100], key=lambda x: cv2.boundingRect(x)[0])
    
    # Assume the first contour is the reference object
    if len(contours) > 0:
        ref_contour = contours[0]
        ref_box = cv2.minAreaRect(ref_contour)
        ref_box = cv2.boxPoints(ref_box)
        ref_box = np.array(ref_box, dtype="int")
        
        # Get the width of the reference object in pixels
        (tl, tr, br, bl) = ref_box
        (tltrX, tltrY) = midpoint(tl, tr)
        (blbrX, blbrY) = midpoint(bl, br)
        ref_pixel_width = np.linalg.norm([tltrX - blbrX, tltrY - blbrY])
        
        # Draw the reference object's bounding box
        cv2.drawContours(frame, [ref_box.astype("int")], -1, (0, 255, 0), 2)
        
        # Loop over the remaining contours
        for contour in contours[1:]:
            box = cv2.minAreaRect(contour)
            box = cv2.boxPoints(box)
            box = np.array(box, dtype="int")
            
            # Get the width of the object in pixels
            (tl, tr, br, bl) = box
            (tltrX, tltrY) = midpoint(tl, tr)
            (blbrX, blbrY) = midpoint(bl, br)
            pixel_width = np.linalg.norm([tltrX - blbrX, tltrY - blbrY])
            
            # Calculate the size of the object in the same units as KNOWN_WIDTH
            size = (pixel_width / ref_pixel_width) * KNOWN_WIDTH
            
            # Draw the object's bounding box
            cv2.drawContours(frame, [box.astype("int")], -1, (0, 0, 255), 2)
            cv2.putText(frame, "{:.2f} units".format(size), (int(tltrX - 15), int(tltrY - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.65, (255, 255, 255), 2)
    
    # Display the resulting frame
    cv2.imshow('Frame', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()
